In [7]:
# Liturgický kalendár: 365 dní, "one page".
# zobrazenie ako "report".

In [8]:
import pandas as pd
from bs4 import BeautifulSoup, Comment
import pickle

In [9]:
# Load the dictionary from the pickle file
with open('dict_of_strings.pickle', 'rb') as file:
    dict_of_strings = pickle.load(file)

In [10]:
with open('mustra_r.html', mode='r', encoding='utf-8') as m:
    mustra = m.read()
    
soup1 = BeautifulSoup(mustra,'lxml')

my_table = soup1.new_tag('table') 
soup1.body.append(my_table)

In [11]:
for date_string, file_content in dict_of_strings.items():

    # týždne sú oddelené hrubšími čiarami, jednotlivé týždne sú zabalené v <tbody> elementoch
    current_date = pd.to_datetime(date_string)
    if (date_string == '2023-01-01') or current_date.day_name('sk_SK')=='Pondelok':
        tbody_here = soup1.new_tag('tbody') # create new tbody here
        soup1.table.append(tbody_here)

    new_table_row = soup1.new_tag('tr')
    tbody_here.append(new_table_row)

    ### soup2
    soup2 = BeautifulSoup(file_content[56:],'lxml') # [56:] , čiže načítavam až od druhého riadku. lebo súbory začínajú ako XML, čo vyhadzovalo zbytočný warning zo strany Parsera.
    
    # h1
    # mažeme link, aj nadbytočný <span> :
    soup2.find('a', class_='lcMY').unwrap() # alebo: soup2.h1.a.unwrap() #  lcMY
    soup2.find('span', class_='lcMD').unwrap() # lcMD
    span_lcDMY = soup2.find('span', class_='lcDMY') # lcDMY
    span_lcDMY.smooth()
    span_lcDMY.string = span_lcDMY.string[:-5] # mažeme rok 2023 z textu 
    # toto nešlo: span_lcDMY.string = span_lcDMY.string.removesuffix(' 2023') # 

    td1 = soup1.new_tag('td', id=date_string) # id='2023-01-01' napríklad. využije sa pri naDnes navigácii
    td1.append( soup2.find('h1', class_='datum') ) # vkladáme hlavičku (dátum)

    for elem in soup2.find_all('div', class_='lcHEAD'): # vkladáme lcHEAD
        h3 = elem.find('h3')
        if h3 and h3.get_text().startswith('Vigília'): # ak ide o vigíliu, nezapisujeme do tabulky 
            pass # also "continue" could be.
        else:                    
            elem.find('div', class_='lcHEADinfo').extract() # mažeme súradnice
            td1.append(elem)        
    
    new_table_row.append(td1) # <tr><td>    

In [12]:
# final cleaning (size reducing)

# removing html comments <!-- -->
comments = soup1.find_all(string = lambda x: isinstance(x, Comment))
for comment in comments:
    comment.extract()

# soup1.smooth() # optionally

with open('onepage_report.html', mode='w', encoding='utf-8') as f_out:
    f_out.write(str(soup1))

In [ ]:
# done.